In [2]:
import numpy as np
import pandas as pd
import math as m
from math import sin as s
from math import cos as c
from math import acos
from math import asin
from math import atan

### REVERSE ANALYSIS FOR Cincinnati Milacron T3-776

In [3]:
### MECHANISM PARAMETERS
aL = [0,44,0,0,0] ## in inches
apL = [90,0,90,61,61] ## in degrees
S = [0,0,55,0] ## S2 through S4, in inches
## all joint angles variable

In [4]:
def R2D(val):
    val = val*(180/m.pi)
    return val

In [5]:
def D2R(val):
    val = val*(m.pi/180)
    return val

In [6]:
def fwdAnalysis(A_vec,alpha_vec,S_vec,theta_rad):
    # A_vec is a 1xN vector that contains link length (a12 through a56)
    # alpha_vec is a 1xN vector that contains twist angles (alpha12 through alpha56)
    # S_vec is a 1xN vector that contains joint offset (S2 through S5)
    # theta_rad contains the given joint angles in radians where theta_rad[0] is phi_1

    i=0
    T_F1 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,0,
                     m.sin(theta_rad[i]),m.cos(theta_rad[i]),0,0,
                     0,0,1,0,
                     0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j=0
    T_12 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
            
        ]).reshape(4,4)

    i+=1
    j+=1
    T_23 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j+=1
    T_34 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j+=1
    T_45 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j+=1
    T_56 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    T_F6 = T_F1@T_12@T_23@T_34@T_45@T_56

    return T_F6

In [7]:
def closeTheLoop(P_6Tool,P_FTool,S_F6,a_F67):
    
    # P_6Tool = P_6Tool#.reshape(3,1)
    # P_FTool = P_FTool#.reshape(3,1)
    # S_F6 = S_F6#.reshape(3,1)
    # a_F67 = a_F67#.reshape(3,1)

    P_F6org = P_FTool - (np.dot(P_6Tool,np.array([1,0,0]))*a_F67 + np.dot(P_6Tool,np.array([0,1,0]))*np.cross(S_F6,a_F67) + np.dot(P_6Tool,np.array([0,0,1]))*S_F6)

    a_67 = 0
    alpha_67 = 90
    x_FF = np.array([1,0,0])#.reshape(3,1)

    S_F7 = np.cross(a_F67,S_F6)
    S_F1 = np.array([0,0,1])#.reshape(3,1)

    c_71 = np.dot(S_F7,S_F1)
    alpha_71 = m.acos(c_71)*(180/m.pi)
    

    ## special case 1
    if np.abs(c_71) == 1:
        S_7 = 0
        s_71 = 0

        S_1 = np.dot(-P_F6org,S_F1)

        a_71 = np.abs(-P_F6org + S_1*S_F1)

        ## special case 2
        if a_71 == 0:
            
            a_F71 = a_F67
            t_7 = 0
            a_71 = a_67
            cg_1 = np.dot(a_F71,x_FF)
            sg_1 = np.dot(np.cross(a_F71,x_FF),S_F1)
            g_1 = m.asin(sg_1)*(180/m.pi)

            return a_71,alpha_71,S_7,t_7,S_1,g_1
    
        a_F71 = (-P_F6org + S_1*S_F1)/a_71

        c7 = np.dot(a_F67,a_F71)
        s7 = np.dot(np.cross(a_F67,a_F71),S_F7)
        t_7 = m.acos(c7)*(180/m.pi)

        cg_1 = np.dot(a_F71,x_FF)
        sg_1 = np.dot(np.cross(a_F71,x_FF),S_F1)
        g_1 = m.asin(sg_1)*(180/m.pi)

        return a_71,alpha_71,S_7,t_7,S_1,g_1

    a_F71 = np.cross(S_F7,S_F1)/np.linalg.norm(np.cross(S_F7,S_F1))

    s_71 = np.dot((np.cross(S_F7,S_F1)),a_F71)

    c7 = np.dot(a_F67,a_F71)
    s7 = np.dot(np.cross(a_F67,a_F71),S_F7)
    t_7 = m.acos(c7)*(180/m.pi)
    

    cg_1 = acos(np.dot(a_F71,x_FF))
    sg_1 = asin(np.dot(np.cross(a_F71,x_FF),S_F1))


    
    g_c = [cg_1,-cg_1,sg_1,(m.pi)-sg_1]

    for k in range(len(g_c)):
        if len([i for i,j in enumerate(g_c) if round(j,0) == round(g_c[k],0)]) > 1:
            g_1 = g_c[k]


    S_7 = np.dot((np.cross(S_F1,P_F6org)),a_F71)/s_71
              
    a_71 = np.dot(np.cross(P_F6org,S_F1),S_F7)/s_71
    


    S_1 = np.dot(np.cross(P_F6org,S_F7),a_F71)/s_71
       
    return a_67, alpha_67, a_71,alpha_71,S_7,t_7,S_1,g_1

In [288]:
def reverseAnalysis(aL,apL,S,S6,P6tool,PFtool,SF6,aF67):
    ## Free choices, Tool Params, CTL
    a67, ap67, a71,ap71,S7,t7,S1,g1 = closeTheLoop(P6tool,PFtool,SF6,aF67)
    ap67 = D2R(ap67)
    ap71 = D2R(ap71)
    t7 = D2R(t7)
    g1 = D2R(g1)

    a12, a23, a34, a45, a56 = aL
    ap12, ap23, ap34, ap45, ap56 = apL
    S2,S3,S4,S5 = S

    ## ANGLE TREE BRANCHES
    b1 = [] ## 7, 1a, 3a, 2a, 5a, 4a
    b2 = [] ## 7, 1a, 3a, 2a, 5b, 4b
    b3 = [] ## 7, 1a, 3b, 2b, 5c, 4c
    b4 = [] ## 7, 1a, 3b, 2b, 5d, 4d
    b5 = [] ## 7, 2a, 3c, 2c, 5e, 4e
    b6 = [] ## 7, 2a, 3c, 2c, 5f, 4f
    b7 = [] ## 7, 2a, 3d, 2d, 5g, 4g
    b8 = [] ## 7, 2a, 3d, 2d, 5h, 4h
    tree = [b1,b2,b3,b4,b5,b6,b7,b8]

    # for i in tree:
    #     i.append(t7)


    ## THETA 1 (2 UNIQUE SOLUTIONS)
    t1t = atan(((S6 * c(t7) * c(ap71)) + (S7 * s(ap71)) ) / ( (S6 * s(t7)) + a71 ))

    t1 = [t1t,t1t-(m.pi)]
    p1 = [t1[0]-g1,t1[1]-g1]
    t1 = list(round(loc,4) for loc in t1)
    for i,j in enumerate(tree):
        if i>3:
            k=1
            j.append(t1[1])
        else:
            k=0
            j.append(t1[0])
   

    ## THETA 3 (4 UNIQUE SOLUTIONS) [DOUBLED]
    t3=[]
    for i in range(len(t1)):
        X1 = s(ap71) * s(t1[i])
        Y1 = -( (s(ap12) * c(ap71)) + (c(ap12) * s(ap71) * c(t7)) )
        X7 = s(ap67) * s(t7)
        Y7 = -( (s(ap71) * c(ap67)) + (c(ap71) * s(ap67) * c(t7)) )
        Z7 = ( c(ap71) * c(ap67) ) - ( s(ap71) * s(ap67) * c(t7) )
        X71 = (X7 * c(t1[i])) + (Y7 * s(t1[i]))
        Y71 = c(ap12) * ((X7 * s(t1[i])) + (Y7 * c(t1[i]))) - (s(ap12) * Z7)

        A3 = (-S6 * X71) - (S7 * X1) - (a71 * c(t1[i]))
        B3 = -S1 + (S6 * Y71) + (S7 * Y1)

        t3s = asin((A3**2 + B3**2 - a23**2 - S4**2) / (2 * a23 * S4))

        t3.append(t3s)
        if t3s<1:
            t3s = -t3s
        t3.append(t3s-m.pi)
    t3 = list(round(loc,4) for loc in t3)
    for i,j in enumerate(tree):
        if i<=1:
            j.append(t3[0])
        if i>1 and i<4:
            j.append(t3[1])
        if i>3 and i<6:
            j.append(t3[2])
        if i>5:
            j.append(t3[3])


    ## THETA 2 (4 UNIQUE SOLUTIONS)
    t2 = []
    for i in range(len(t1)):
        for j in range(len(t3)):
            if (i==0 and j==0) or (i==0 and j==1) or (i==1 and j==2) or (i==1 and j==3):
                X1 = s(ap71) * s(t1[i])
                Y1 = -( (s(ap12) * c(ap71)) + (c(ap12) * s(ap71) * c(t7)) )
                X7 = s(ap67) * s(t7)
                Y7 = -( (s(ap71) * c(ap67)) + (c(ap71) * s(ap67) * c(t7)) )
                Z7 = ( c(ap71) * c(ap67) ) - ( s(ap71) * s(ap67) * c(t7) )
                X71 = (X7 * c(t1[i])) + (Y7 * s(t1[i]))
                Y71 = c(ap12) * ((X7 * s(t1[i])) + (Y7 * c(t1[i]))) - (s(ap12) * Z7)
                A3 = (-S6 * X71) - (S7 * X1) - (a71 * c(t1[i]))
                B3 = -S1 + (S6 * Y71) + (S7 * Y1)
                A11 = a23 + (S4 * s(t3[j]))
                A12 = S4 * c(t3[j])
                A21 = -S4 * c(t3[j])
                A22 = a23 + (S4 * s(t3[j]))
                B11 = A3
                B21 = B3

                A = np.array([[A11, A12],[A21, A22]

                ])
                
                B = np.array([B11,B21

                ])

                t2c, t2s = np.linalg.solve(A,B)

                t2c = acos(t2c)
                t2s = asin(t2s)


                if t2s<0:
                    t2sc = (-m.pi)-t2s
                if t2s>0:
                    t2sc = m.pi-t2s


                t2_calc = [t2c,-t2c,t2s,t2sc]
                for f in range(len(t2_calc)):
                        if len([g for g,h in enumerate(t2_calc) if round(h,2) == round(t2_calc[f],2)]) > 1:
                            t2.append(t2_calc[f])
                            break
    for i,j in enumerate(tree):
        if i<=1:
            j.append(t2[0])
        if i>1 and i<4:
            j.append(t2[1])
        if i>3 and i<6:
            j.append(t2[2])
        if i>5:
            j.append(t2[3])


    # ## THETA 5 (8 UNIQUE SOLUTIONS) [DOUBLED]
    t5 = []
    for i in range(len(t1)):
        for j in range(len(t3)):
            if (i==0 and j==0) or (i==0 and j==1) or (i==1 and j==2) or (i==1 and j==3):
                X7 = s(ap67) * s(t7)
                Y7 = -( (s(ap71) * c(ap67)) + (c(ap71) * s(ap67) * c(t7)) )
                Z7 = ( c(ap71) * c(ap67) ) - ( s(ap71) * s(ap67) * c(t7) )

                X71 = (X7 * c(t1[i])) - (Y7 * s(t1[i]))
                Y71 = (c(ap12) * ((X7 * s(t1[i]))) + (Y7 * c(t1[i]))) - (s(ap12) * Z7)
                Z71 = (s(ap12) * ((X7 * s(t1[i]))) + (Y7 * c(t1[i]))) + (c(ap12) * Z7)

                X712 = (X71 * c(t2[j])) - (Y71 * s(t2[j]))
                Y712 = (c(ap23) * ((X71 * s(t2[j])) + (Y71 * c(t2[j])))) - (s(ap23) * Z71)
                Z712 = (s(ap23) * ((X71 * s(t2[j])) + (Y71 * c(t2[j])))) + (c(ap23) * Z71)

                Z7123 = (s(ap34) * ((X712 * s(t3[j])) + (Y712 * c(t3[j])))) + (c(ap34) * Z712)

                t5.append( acos( ((c(ap56) * c(ap45)) - Z7123) / (s(ap56) * s(ap45)) ))
                t5.append( - acos( (c(ap56) * c(ap45) - Z7123) / (s(ap56) * s(ap45)) ))
        
    for i,j in enumerate(tree):
        j.append(t5[i])

    # ## THETA 4 (8 UNIQUE SOLUTIONS)
    t4 = []
    for i in range(len(t1)):
        for j in range(len(t3)):
            for k in range(len(t5)):
                if (i==0 and j==0 and k==0) or (i==0 and j==0 and k==1) or (i==0 and j==1 and k==2) or (i==0 and j==1 and k==3) or (i==1 and j==2 and k==4) or (i==1 and j==2 and k==5) or (i==1 and j==3 and k==6) or (i==1 and j==3 and k==7):

                    X7 = s(ap67) * s(t7)
                    Y7 = -( (s(ap71) * c(ap67)) + (c(ap71) * s(ap67) * c(t7)) )
                    Z7 = ( c(ap71) * c(ap67) ) - ( s(ap71) * s(ap67) * c(t7) )


                    X71 = (X7 * c(t1[i])) - (Y7 * s(t1[i]))
                    Y71 = (c(ap12) * ((X7 * s(t1[i])) + (Y7 * c(t1[i])))) - (s(ap12) * Z7)
                    Z71 = (s(ap12) * ((X7 * s(t1[i])) + (Y7 * c(t1[i])))) + (c(ap12) * Z7)

                    X712 = (X71 * c(t2[j])) - (Y71 * s(t2[j]))
                    Y712 = (c(ap23) * ((X71 * s(t2[j])) + (Y71 * c(t2[j])))) - (s(ap23) * Z71)
                    Z712 = (s(ap23) * ((X71 * s(t2[j])) + (Y71 * c(t2[j])))) + (c(ap23) * Z71)

                    X7123 = (X712  * c(t3[j])) - (Y712 * s(t3[j]))
                    Y7123 = c(ap34) * ((X712 * s(t3[j])) + (Y712 * c(t3[j]))) - (s(ap34) * Z712)

                    Xbar5 = s(ap56) * s(t5[k])
                    Ybar5 = -( (s(ap45) * c(ap56)) + (c(ap45) * s(ap56) * c(t5[k])) )

                    A11 = Xbar5
                    A12 = -Ybar5
                    A21 = Ybar5
                    A22 = Xbar5
                    B11 = X7123
                    B21 = -Y7123

                    A = np.array([[A11, A12],[A21, A22]

                    ])
                    
                    B = np.array([B11,B21

                    ])

                    t4c, t4s = np.linalg.solve(A,B)

                    t4c = acos(t4c)
                    t4s = asin(t4s)

                    if t4s<0:
                        t4sc = (-m.pi)-t4s
                    if t4s>0:
                        t4sc = m.pi-t4s


                    t4_calc = [t4c,-t4c,t4s,t4sc]
                    
                    for f in range(len(t4_calc)):
                                if len([g for g,h in enumerate(t4_calc) if round(h,2) == round(t4_calc[f],2)]) > 1:
                                    t4.append(t4_calc[f])
                                    break

    for i,j in enumerate(tree):
        j.append(t4[i])
    # ## THETA 6 (8 UNIQUE SOLUTIONS)
    t6 = []
    for i in range(len(t1)):
        for j in range(len(t3)):
            for k in range(len(t4)):
                if (i==0 and j==0 and k==0) or (i==0 and j==0 and k==1) or (i==0 and j==1 and k==2) or (i==0 and j==1 and k==3) or (i==1 and j==2 and k==4) or (i==1 and j==2 and k==5) or (i==1 and j==3 and k==6) or (i==1 and j==3 and k==7):
                    Xbar4 = s(ap45) * s(t4[k])
                    Ybar4 = -( (s(ap34) * c(ap45)) + ( c(ap34) * s(ap45) * c(t4[k])) )
                    Zbar4 = ( (c(ap34) * c(ap45)) - ( s(ap34) * s(ap45) * c(t4[k])) )

                    X43 = (Xbar4 * c(t3[j])) - (Ybar4 * s(t3[j]))
                    Y43 = c(ap23) * ( (Xbar4 * s(t3[j])) + (Ybar4 * c(t3[j])) ) - ( s(ap23)*Zbar4 )
                    Z43 = s(ap23) * ( (Xbar4 * s(t3[j])) + (Ybar4 * c(t3[j])) ) + ( c(ap23)*Zbar4 )

                    X432 = (X43 * c(t2[j])) - (Y43 * s(t2[j]))
                    Y432 = c(ap12) * ( (X43 * s(t2[j])) + (Y43 * c(t2[j])) ) - ( s(ap12)*Z43 )
                    Z432 = s(ap12) * ( (X43 * s(t2[j])) + (Y43 * c(t2[j])) ) + ( c(ap12)*Z43 )

                    X4321 = (X432 * c(t1[i])) - (Y432 * s(t1[i]))
                    Y4321 = c(ap71) * ( (X432 * s(t1[i])) + (Y432 * c(t1[i])) ) - ( s(ap71)*Z432 )
                    Z4321 = s(ap71) * ( (X432 * s(t1[i])) + (Y432 * c(t1[i])) ) + ( c(ap71)*Z432 )

                    X43217 = (X4321 * c(t7) - (Y4321 * s(t7)))
                    Y43217 = (c(ap67) * ( (X4321 * s(t7)) + (Y4321 * c(t7)) )) - ( s(ap67)*Z4321 )

                    t6c = acos( Y43217 / s(ap56) )
                    t6s = asin( X43217 / s(ap56) )

                    if t6s<0:
                        t6sc = (-m.pi)-t6s
                    if t6s>0:
                        t6sc = m.pi-t6s


                    t6_calc = [t6c,-t6c,t6s,t6sc]

                    for f in range(len(t6_calc)):
                            if len([g for g,h in enumerate(t6_calc) if round(h,2) == round(t6_calc[f],2)]) > 1:
                                t6.append(t6_calc[f])
                                break
    for i,j in enumerate(tree):
        j.append(t6[i])

    return tree

In [322]:
## UFID is 57758617 -> Reverse Analysis on Cincinnati Milacron T3-776

## 
P6tool = np.array([5,3,7])
PFtool = np.array([55,33,23])
SF6 = np.array([1,0,0])
aF67 = np.array([0,0.707,0.707])


# link length in INCHES
aL = np.array([0,44,0,0,0

])

# twist angle in DEGREES
apL = np.array([90,0,90,61,61
    
])*(m.pi/180) ## convert to radians ##


# joint offest in INCHES
S6 = 6
S = np.array([0,0,55,0
              
]) # PICK S6

# joint angle in RADIANS... 
angles = reverseAnalysis(aL,apL,S,S6,P6tool,PFtool,SF6,aF67)

## visual output
df = pd.DataFrame(angles,index=['A','B','C','D','E','F','G','H'],columns=['ϕ1','θ3','θ2','θ5','θ4','θ6']).apply(R2D).apply(round,ndigits=2)
df = df.reindex(['ϕ1','θ2','θ3','θ4','θ5','θ6'],axis=1)
df
## answer matches book

,ϕ1,θ2,θ3,θ4,θ5,θ6
A,36.96,84.35,-23.08,70.87,127.50,100.82
B,36.96,84.35,-23.08,-136.03,-127.50,-106.07
C,36.96,-47.83,-156.91,163.41,97.46,-6.46
D,36.96,-47.83,-156.91,-62.68,-97.46,127.44
E,-143.04,-132.17,-23.08,-16.59,97.47,-6.46
F,-143.04,-132.17,-23.08,117.32,-97.47,127.45
G,-143.04,95.65,-156.91,-109.13,127.50,100.82
H,-143.04,95.65,-156.91,43.97,-127.50,-106.07
